In [1]:
from datasets import load_dataset
from tqdm import tqdm
from datasets import load_metric
import pandas as pd

c:\Users\39331\anaconda3\envs\deep_nlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
'''
file_path points to a fie in this folder with summaries separated by '\n' (1 summary/row). 
The rouge metric is evaluated over the test set of samsum dataset. 
'''

file_path = './tmp_result_EMOJI_M1.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    candidates = file.read().split('\n')

candidates = candidates[:-1] # the last one is an empy new line, drop it.

samsum = load_dataset('samsum')
samsum_test_summary = samsum['test']['summary']

Found cached dataset samsum (C:/Users/39331/AppData/Roaming/SPB_16.6/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)
100%|██████████| 3/3 [00:00<00:00, 751.80it/s]


In [3]:
metric = load_metric('../utils/rouge.py')

results_v2=[]
for i in tqdm(range(len(candidates))):
    
    r = metric.compute(predictions=[candidates[i]], references=[samsum_test_summary[i]])
    r = {key: value.mid.fmeasure * 100 for key, value in r.items()}
    
    results_v2.append(r)

h:\Il mio Drive\2_1_Deep_NLP\trials\rouge_clean_emoji_M1.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  from datasets import load_dataset
100%|██████████| 819/819 [01:55<00:00,  7.08it/s]


In [4]:
res_v2 = pd.DataFrame([samsum_test_summary, candidates, results_v2]).T
res_v2.columns = ['samsum_test_summary', 'candidates', 'results']
res_v2 = res_v2['results'].apply(pd.Series).merge(res_v2, left_index=True, right_index=True).drop('results', axis=1)

res_v2.to_excel('./res_v2_rouge_scores_EMOJI_M1.xlsx', encoding='utf-8')